Comentario:
Esta notebook sería la demostración de cómo entrenamos ResNet-50 en SIPaKMED, pero es importante comentar que esta última versión no es de la cual obtuvimos los pesos que utilizamos en el informe. En esta última aplicamos augmentaciones y se entrenó por más de 10 épocas. Los pesos aprendidos en el caso de 10 épocas fueron guardados durante el entrenamiento, pero la notebook en sí no es esta. De todas formas, en general, la implementación es igual.


In [ ]:
from google.colab import drive
import sys
import torch
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Mounted at /content/drive


In [ ]:
import kagglehub


path = kagglehub.dataset_download("prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed")

print("Path to dataset files:", path)

100%|██████████| 6.40G/6.40G [01:01<00:00, 111MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed/versions/1


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
data=[]
labels=[]
height = 64
width = 64

path1 = path + "/im_Dyskeratotic/im_Dyskeratotic/CROPPED/"
path2 = path + "/im_Koilocytotic/im_Koilocytotic/CROPPED/"
path3 = path + "/im_Metaplastic/im_Metaplastic/CROPPED/"
path4 = path + "/im_Parabasal/im_Parabasal/CROPPED/"
path5 = path + "/im_Superficial-Intermediate/im_Superficial-Intermediate/CROPPED/"
pathes = [path1, path2, path3, path4, path5]

i = 0
for p in pathes:
    Class=os.listdir(p)
    for a in Class:
        if(a[-1] == 'p'):
            labels.append(i)
            try:
                image=cv2.imread(p+a)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((height, width))
                data.append(np.array(size_image))

            except AttributeError:
                print(" ")
    i+=1

labels = np.array(labels)
data = np.array(data)

print(data.shape)
print(labels.shape)
print(labels)

s = np.arange(data.shape[0])
np.random.seed(43)
np.random.shuffle(s)
data = data[s]
labels = labels[s]

print(labels)

(4049, 64, 64, 3)
(4049,)
[0 0 0 ... 4 4 4]
[1 1 4 ... 2 4 2]


In [ ]:
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
len(train_images), len(val_images), len(train_labels), len(val_labels)

(3239, 810, 3239, 810)

In [ ]:
import torch
import torchvision.models as models


resnet50 = models.resnet50(pretrained=True)
num_classes = len(set(labels))

resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)
resnet50.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 208MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [ ]:
from torchvision import transforms




import torchvision.transforms as transforms
from torchvision.transforms import functional as F
import random

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a tensor first
    transforms.RandomApply([
        transforms.RandomRotation(degrees=(-90, 90)),  # Rotación aleatoria entre -90° y 90°
        transforms.RandomHorizontalFlip(),  # Espejado horizontal
        transforms.RandomVerticalFlip(),    # Espejado vertical
        transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),  # Zoom aleatorio
    ], p=0.7),  # Aplicar con probabilidad del 70%

    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Variación de color
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Desenfoque Gaussiano
        transforms.Lambda(lambda img: F.adjust_gamma(img, gamma=random.uniform(0.8, 1.2)))  # Ajuste de gamma aleatorio
    ], p=0.3),  # Aplicar con probabilidad del 30%

    transforms.Resize((224, 224)),  # Redimensionar las imágenes al tamaño esperado por ResNet-50
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizar como en ImageNet
])



dataset_train = CustomImageDataset(train_images, train_labels, transform=transform)
dataset_val = CustomImageDataset(val_images, val_labels, transform=transform)

trainloader = DataLoader(dataset_train , batch_size=32, shuffle=True)
valloader = DataLoader(dataset_val, batch_size=32, shuffle=False)

In [ ]:
import torch.optim as optim
import torch.nn as nn

for name, param in resnet50.named_parameters():
    if 'layer3' in name or 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet50.parameters()), lr=0.001)




In [ ]:
resnet50.to(device)
num_epochs = 20
best_acc = 0
for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in trainloader:

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        print(loss)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Época [{epoch + 1}/{num_epochs}], Pérdida: {running_loss / len(trainloader):.4f}, Precisión: {100 * correct / total:.2f}%")

    resnet50.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_images, val_labels in valloader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = resnet50(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()


    val_accuracy = 100 * val_correct / val_total
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        best_weights = resnet50.state_dict()


    print(f"Validación - Pérdida: {val_loss / len(valloader):.4f}, Precisión: {val_accuracy:.2f}%")



<ipython-input-15-63452f6b2ea7>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


tensor(1.5838, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1697, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6248, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1729, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8675, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8148, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8461, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7230, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7682, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8070, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9307, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9046, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8614, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7752, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1247, device='cuda:0', grad_fn=

In [ ]:
torch.save({'model_state_dict':best_weights,}, path_base + "/pesos_modelos/Resnet20_SipakMed_50_epochs_augmentation.pth")